# 회원 탈퇴 예측하기
<br>
Decision Tree (의사결정 트리) 배우기

## 1. 데이터 읽고 집계하기

In [1]:
!ls pyda100_clone/5장

customer_join.csv    use_log_months.csv   제5장_answer.ipynb


In [2]:
import pandas as pd

customer = pd.read_csv("pyda100_clone/5장/customer_join.csv")
uselog_months = pd.read_csv("pyda100_clone/5장/use_log_months.csv")

**왜 2개월만?**  
예측을 위해 이 달과 1개월 전의 이용 이력만으로 데이터를 작성한다.  
과거 6개월분의 데이터로 예측하는 경우, 가입 5개월 이내의 회원 탈퇴는 예측할 수 없다.  
3장에서도 불과 몇 개월만에 그만둔 회원도 많기때문에 6개월분은 의미가 없다.  
따라서 이 달과 1개월 전의 이용횟수를 집계한 데이터를 작성한다.  

In [3]:
year_months = list(uselog_months["연월"].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):  # 왜 연월 '201804' 빼고 할까요?
    tmp = uselog_months.loc[uselog_months["연월"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["연월"]==year_months[i-1]]
    del tmp_before["연월"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

/Users/yoon/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


In [33]:
year_months

[201804,
 201805,
 201806,
 201807,
 201808,
 201809,
 201810,
 201811,
 201812,
 201901,
 201902,
 201903]

## 2. 탈퇴 전월의 탈퇴 고객 데이터를 작성하기

In [4]:
from dateutil.relativedelta import relativedelta

exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])

for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)


/Users/yoon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/yoon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/yoon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.

In [5]:
exit_customer[["exit_date"]]

,exit_date
708,2018-03-30 00:00:00
729,2018-03-30 00:00:00
730,2018-03-30 00:00:00
770,2018-04-30 00:00:00
785,2018-04-30 00:00:00
...,...
4049,2019-01-28 00:00:00
4050,2019-02-28 00:00:00
4068,2019-02-28 00:00:00
4086,2019-02-28 00:00:00


In [6]:
exit_customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,종일,10500,일반,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,주간,7500,일반,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,종일,10500,일반,1.0,1.0,1,1,0,2018-04-30,23,2018-03-30 00:00:00
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,야간,6000,일반,3.5,3.5,4,3,1,2018-05-31,23,2018-04-30 00:00:00
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,종일,10500,일반,3.0,3.0,3,3,0,2018-05-31,23,2018-04-30 00:00:00


In [7]:
exit_customer["연월"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["연월"] = uselog["연월"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "연월"], how="left")

print(len(uselog))

exit_uselog.head()

33851


/Users/yoon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


탈퇴한 회원으로 결합했으므로 결측치 많음  
결측치 제거하면 탈퇴한 회원의 uselog 얻을 수 있음

In [8]:
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()  # 탈퇴한 회원의 uselog

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


## 3. 지속 회원의 데이터 작성하기

In [9]:
conti_customer = customer.loc[customer["is_deleted"]==0]
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))

33851


In [10]:
conti_uselog.isna().sum()

연월                       0
customer_id              0
count_0                  0
count_1               1201
name                  6429
class                 6429
gender                6429
start_date            6429
end_date             33851
campaign_id           6429
is_deleted            6429
class_name            6429
price                 6429
campaign_name         6429
mean                  6429
median                6429
max                   6429
min                   6429
routine_flg           6429
calc_date             6429
membership_period     6429
dtype: int64

In [11]:
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

27422


In [34]:
conti_uselog.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               208
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
dtype: int64

탈퇴 회원이 1104명이므로 27422명의 지속 회원을 전부 사용하면 불균형 데이터가 된다.  
그러므로 지속 회원당 데이터 1개만 갖도록 언더샘플링 한다.  
-> 18년 5월 A 회원과 18년 12월 A 회원 중 하나의 데이터만 선택한다는 뜻

In [12]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
# sample: 무작위 샘플링인데, "frac=1, reset_index(drop=True)" 을 이용해 전체 데이터 셔플 가능. frac<1 이면 frac 비율만큼 샘플링

conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")  
# drop_duplicates: 중복값 처리(unique한 1개의 key만 남기고 나머지 중복은 제거) 

print(len(conti_uselog))
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201812,AS359364,6,2.0,XX,C03,F,2015-10-01,NaN,CA1,...,야간,6000.0,일반,3.750000,4.0,6.0,1.0,1.0,2019-04-30,42.0
1,201807,AS060041,6,8.0,XXXX,C01,M,2017-06-01,NaN,CA1,...,종일,10500.0,일반,6.666667,7.0,9.0,4.0,1.0,2019-04-30,22.0
2,201807,IK067006,8,6.0,XXXXX,C01,M,2018-01-01,NaN,CA1,...,종일,10500.0,일반,7.416667,7.5,10.0,4.0,1.0,2019-04-30,15.0
4,201901,GD869667,7,6.0,XXX,C01,M,2018-06-11,NaN,CA2,...,종일,10500.0,입회비반액할인,8.000000,7.5,12.0,5.0,1.0,2019-04-30,10.0
5,201903,PL520159,5,2.0,XXXX,C01,M,2017-08-01,NaN,CA2,...,종일,10500.0,입회비반액할인,6.083333,5.5,10.0,2.0,1.0,2019-04-30,20.0


In [13]:
# 지속회원과 탈퇴회원 결합
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


/Users/yoon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,calc_date,campaign_id,campaign_name,class,class_name,count_0,count_1,customer_id,end_date,exit_date,...,max,mean,median,membership_period,min,name,price,routine_flg,start_date,연월
0,2019-04-30,CA1,일반,C03,야간,6,2.0,AS359364,NaN,NaN,...,6.0,3.750000,4.0,42.0,1.0,XX,6000.0,1.0,2015-10-01,201812
1,2019-04-30,CA1,일반,C01,종일,6,8.0,AS060041,NaN,NaN,...,9.0,6.666667,7.0,22.0,4.0,XXXX,10500.0,1.0,2017-06-01,201807
2,2019-04-30,CA1,일반,C01,종일,8,6.0,IK067006,NaN,NaN,...,10.0,7.416667,7.5,15.0,4.0,XXXXX,10500.0,1.0,2018-01-01,201807
3,2019-04-30,CA2,입회비반액할인,C01,종일,7,6.0,GD869667,NaN,NaN,...,12.0,8.000000,7.5,10.0,5.0,XXX,10500.0,1.0,2018-06-11,201901
4,2019-04-30,CA2,입회비반액할인,C01,종일,5,2.0,PL520159,NaN,NaN,...,10.0,6.083333,5.5,20.0,2.0,XXXX,10500.0,1.0,2017-08-01,201903


## 4. 예측할 달의 재적 기간을 작성하기

시간적 요소가 들어간 데이터이므로 재적 기간 같은 데이터를 변수로 이용하는것이 좋다.

In [14]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["연월"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])

for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)

predict_data.head()

/Users/yoon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,calc_date,campaign_id,campaign_name,class,class_name,count_0,count_1,customer_id,end_date,exit_date,...,median,membership_period,min,name,price,routine_flg,start_date,연월,period,now_date
0,2019-04-30,CA1,일반,C03,야간,6,2.0,AS359364,NaN,NaN,...,4.0,42.0,1.0,XX,6000.0,1.0,2015-10-01,201812,38,2018-12-01
1,2019-04-30,CA1,일반,C01,종일,6,8.0,AS060041,NaN,NaN,...,7.0,22.0,4.0,XXXX,10500.0,1.0,2017-06-01,201807,13,2018-07-01
2,2019-04-30,CA1,일반,C01,종일,8,6.0,IK067006,NaN,NaN,...,7.5,15.0,4.0,XXXXX,10500.0,1.0,2018-01-01,201807,6,2018-07-01
3,2019-04-30,CA2,입회비반액할인,C01,종일,7,6.0,GD869667,NaN,NaN,...,7.5,10.0,5.0,XXX,10500.0,1.0,2018-06-11,201901,6,2019-01-01
4,2019-04-30,CA2,입회비반액할인,C01,종일,5,2.0,PL520159,NaN,NaN,...,5.5,20.0,2.0,XXXX,10500.0,1.0,2017-08-01,201903,19,2019-03-01


## 5. 결측치 제거하기

In [15]:
predict_data.isna().sum()

calc_date               0
campaign_id             0
campaign_name           0
class                   0
class_name              0
count_0                 0
count_1               260
customer_id             0
end_date             2842
exit_date            2842
gender                  0
is_deleted              0
max                     0
mean                    0
median                  0
membership_period       0
min                     0
name                    0
price                   0
routine_flg             0
start_date              0
연월                      0
period                  0
now_date                0
dtype: int64

In [16]:
predict_data = predict_data.dropna(subset=["count_1"])
predict_data.isna().sum()

calc_date               0
campaign_id             0
campaign_name           0
class                   0
class_name              0
count_0                 0
count_1                 0
customer_id             0
end_date             2634
exit_date            2634
gender                  0
is_deleted              0
max                     0
mean                    0
median                  0
membership_period       0
min                     0
name                    0
price                   0
routine_flg             0
start_date              0
연월                      0
period                  0
now_date                0
dtype: int64

## 6. 카테고리형 변수 처리하기

In [17]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,일반,야간,F,2.0,1.0,38,0.0
1,일반,종일,M,8.0,1.0,13,0.0
2,일반,종일,M,6.0,1.0,6,0.0
3,입회비반액할인,종일,M,6.0,1.0,6,0.0
4,입회비반액할인,종일,M,2.0,1.0,19,0.0


In [18]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F,gender_M
0,2.0,1.0,38,0.0,1,0,0,1,0,0,1,0
1,8.0,1.0,13,0.0,1,0,0,0,1,0,0,1
2,6.0,1.0,6,0.0,1,0,0,0,1,0,0,1
3,6.0,1.0,6,0.0,0,0,1,0,1,0,0,1
4,2.0,1.0,19,0.0,0,0,1,0,1,0,0,1


**get_dummies 참고할 것**  
gender_F 가 1이면 반드시 gender_M 은 0이다.  
class_name_야간도 종일,주간이 0일때만 1이다. (dependent)  
이런 변수들을 하나씩 지워줘야 한다

In [19]:
del predict_data["campaign_name_일반"]
del predict_data["class_name_야간"]
del predict_data["gender_M"]
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_F
0,2.0,1.0,38,0.0,0,0,0,0,1
1,8.0,1.0,13,0.0,0,0,1,0,0
2,6.0,1.0,6,0.0,0,0,1,0,0
3,6.0,1.0,6,0.0,0,1,1,0,0
4,2.0,1.0,19,0.0,0,1,1,0,0


## 7. Decision Tree 사용해서 예측모델 구축하기

In [20]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data["is_deleted"]==1]
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 1.
 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1.
 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1.
 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1.
 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1.
 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0.
 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1.
 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1.
 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1.
 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0.

In [21]:
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head()

,y_test,y_pred
1007,1.0,1.0
593,1.0,1.0
1534,0.0,0.0
119,1.0,1.0
1457,0.0,0.0


## 8. 예측 모델 평가하고 모델 튜닝하기

In [22]:
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.870722433460076


In [23]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.870722433460076
0.9759188846641318


In [24]:
X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.903041825095057
0.9233206590621039


## 9. 모델에 기여한 변수 확인하기

In [25]:
importance = pd.DataFrame({"feature_names":X.columns, "coefficient":model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.340804
1,routine_flg,0.138959
2,period,0.519472
3,campaign_name_입회비무료,0.000000
4,campaign_name_입회비반액할인,0.000000
5,class_name_종일,0.000000
6,class_name_주간,0.000106
7,gender_F,0.000659


## 10. 회원 탈퇴 예측하기

In [29]:
count_1 = 3
routine_flg = 1
period = 10
campaign_name = "입회비무료"
class_name = "종일"
gender = "M"

In [30]:
if campaign_name == "입회비반값할인":
    campaign_name_list = [1, 0]
elif campaign_name == "입회비무료":
    campaign_name_list = [0, 1]
elif campaign_name == "일반":
    campaign_name_list = [0, 0]
if class_name == "종일":
    class_name_list = [1, 0]
elif class_name == "주간":
    class_name_list = [0, 1]
elif class_name == "야간":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

In [32]:
input_data

[3, 1, 10, 0, 1, 1, 0, 0]

In [31]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))  # 각 지속/탈퇴에 관한 확률

[1.]
[[0.01367188 0.98632812]]
